In [1]:
from sklearn.datasets import make_regression
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats
import math 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm1
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [2]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',',decimal=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('csgo_round_snapshots.csv')
data = dataset.values
dataset.describe()


,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
count,122410,122410,122410,122410,122410,122410,122410,122410,122410,122410,...,122410,122410,122410,122410,122410,122410,122410,122410,122410,122410
unique,10782,33,34,8,2,501,502,415,477,1360,...,7,7,7,6,4,4,6,4,3,2
top,175.0,0.0,0.0,de_inferno,False,500.0,500.0,500.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T
freq,11226,11683,12161,23811,108726,68258,63759,15886,17722,2434,...,41241,51955,53579,69051,120058,116893,59004,119128,119372,62406


In [3]:
dataset = pd.read_csv('csgo_round_snapshots.csv')

In [4]:
#no tenim null data

In [5]:
#dataset.isnull().sum()

In [6]:
# a les dades hi ha 3 columnes formades per string, com ens indiquen tipus de dades, podem fer servir ints

In [7]:
t=[]
for i in dataset.columns:
    t.append(dataset[i].nunique())
temp =[]
for i in range(len(t)):
    if t[i]==1:
        temp.append(i)
        print(dataset.columns[i])

ct_weapon_bizon
ct_weapon_g3sg1
t_weapon_m249
ct_weapon_negev
ct_weapon_r8revolver
ct_weapon_sawedoff


In [8]:
for i in temp[::-1]:
    dataset=dataset.drop([dataset.columns[i]],axis=1)
print(dataset.columns.shape)

(91,)


In [9]:
dataset.columns.shape
dataset.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.000,0.000,0.000,de_dust2,False,500.000,500.000,0.000,0.000,4000.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,CT
1,156.030,0.000,0.000,de_dust2,False,500.000,500.000,400.000,300.000,600.000,...,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,CT
2,96.030,0.000,0.000,de_dust2,False,391.000,400.000,294.000,200.000,750.000,...,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,CT
3,76.030,0.000,0.000,de_dust2,False,391.000,400.000,294.000,200.000,750.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,CT
4,174.970,1.000,0.000,de_dust2,False,500.000,500.000,192.000,0.000,18350.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,CT


In [10]:
#convertir strings en ints

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
label_encoder = sklearn.preprocessing.LabelEncoder() 
dataset['map'] = label_encoder.fit_transform(dataset['map'])
dataset['bomb_planted'] = label_encoder.fit_transform(dataset['bomb_planted'])
dataset['round_winner'] = label_encoder.fit_transform(dataset['round_winner'])

In [12]:
#dataset.head

In [13]:
#afegir variable de diferencia entre jugadors vius
dataset['difference_between_players_alive']=dataset['t_players_alive']-dataset['ct_players_alive']
dataset['difference_between_players_health']=dataset['t_health']-dataset['ct_health']
dataset['difference_between_players_armor']=dataset['t_armor']-dataset['ct_armor']


In [14]:
#dataset['difference_between_players_alive']
#dataset=(dataset-dataset.mean())/dataset.std()

In [15]:
#dataset = dataset.reset_index()
y = dataset.filter(['round_winner'])
X =dataset.drop(['round_winner'],axis=1)
X=(X-X.mean())/X.std()


In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.2,random_state =42)

In [17]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l1"), max_features=10)
embeded_lr_selector.fit(X, y.values.ravel())

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

C:\Users\oriol\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


10 selected features


In [18]:
embeded_lr_feature

['bomb_planted',
 't_helmets',
 't_weapon_ak47',
 'ct_weapon_awp',
 'ct_weapon_m4a4',
 't_weapon_sg553',
 't_grenade_smokegrenade',
 'difference_between_players_alive',
 'difference_between_players_health',
 'difference_between_players_armor']

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.2,random_state =42)
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(X_train, np.ravel(Y_train,order='C'))
sel_.get_support()
X_train = pd.DataFrame(X_train)
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

total features: 93
selected features: 92
features with coefficients shrank to zero: 1


In [20]:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats
X_train = sel_.transform(X_train)
X_test = sel_.transform(X_test)
X_train.shape, X_test.shape

((97928, 92), (24482, 92))

In [21]:
#X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.2,random_state =42)
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2', solver='liblinear'))
sel_.fit(X_train, np.ravel(Y_train,order='C'))
sel_.get_support()
X_train = pd.DataFrame(X_train)
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print(selected_feat)
print('features with coefficients shrank to zero: {}'.format(np.sum(sel_.estimator_.coef_ == 0)))

total features: 92
selected features: 30
Int64Index([ 4,  5,  7,  8,  9, 10, 12, 15, 16, 17, 18, 20, 21, 27, 31, 34, 36,
            39, 46, 57, 58, 59, 69, 80, 82, 83, 86, 89, 90, 91],
           dtype='int64')
features with coefficients shrank to zero: 0


In [22]:
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
print(removed_feats)
X_train_selected = sel_.transform(X_train)
X_test_selected = sel_.transform(X_test)
X_train_selected.shape, X_train_selected.shape

Int64Index([], dtype='int64')


((97928, 30), (97928, 30))

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
# Train the classifier
clf.fit(X_train_selected,np.ravel(Y_train,order='C'))
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(X_test_selected)
# View The Accuracy Of Our Selected Feature Model
print("Accuracy",accuracy_score(Y_test, y_pred))

Accuracy 0.8563434359937914


In [24]:
from sklearn.metrics import classification_report,accuracy_score
X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.2,random_state =42)
model_3 = RandomForestClassifier()
model_3.fit(X_train,Y_train.values.ravel())
pred_3 = model_3.predict(X_test)
cr3 = classification_report(Y_test.values.ravel(),pred_3)
print(cr3)

C:\Users\oriol\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.82      0.88      0.85     12004
           1       0.88      0.81      0.84     12478

    accuracy                           0.85     24482
   macro avg       0.85      0.85      0.85     24482
weighted avg       0.85      0.85      0.85     24482



Prueba de Torch

In [25]:
import pandas as pd
import torch.utils.data as data_utils

import pandas as pd
import torch
import random
import argparse
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns



In [26]:
batch_size = 100        # number of samples during training
test_batch_size = 1000  # number of samples for test 
epochs = 5              # number of epochs to train (default: 14)
lr = 0.01               # learning rate (default: 1.0)
gamma = 0.7             # Learning rate step gamma (default: 0.7)

no_cuda = True          # disables CUDA training
dry_run = False         # quickly check a single pass
seed = 1                # random seed (default: 1)
log_interval = 50       # how many batches to wait before logging training status
save_model = False     # For Saving the current Model


# Check if cuda is available
use_cuda = not no_cuda and torch.cuda.is_available()
print(f"USING CUDA: {use_cuda}")
torch.manual_seed(seed)

# define the device where to compute (cpu or gpu)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.w = torch.nn.Linear(30, 1)      # creem un model amb 784 entrades i 1 sortida

    def forward(self, x):                    # x:  bsx28x28
        x = torch.flatten(x, 1)              # flatten converteix una matriu en un array  x-> 28x28 -> 784x1
        x = self.w(x)                        # aplica els pesos sum(w*x) ->  784x1 -> 1
        x = torch.sigmoid(x)            # aplica la sigmoid  <- SVM la fa servir??
        return torch.flatten(x, 0)      # return bs probs
class LogisticLoss(torch.nn.modules.Module):
    def __init__(self):
        super(LogisticLoss, self).__init__()

    def forward(self, outputs, labels):
        batch_size = outputs.size()[0]
        outputs = (outputs * 2) - 1
        labels = (labels * 2) - 1  # labels -> 1 or -1
        return torch.sum(torch.log(1 + torch.exp(-(outputs.t() * labels)))) / batch_size

class HingeLoss(torch.nn.modules.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()

    def forward(self, outputs, labels):
        batch_size = outputs.size()[0]
        outputs = (outputs * 2) - 1
        labels = (labels * 2) - 1   # labels -> 1 or -1
        return torch.sum(torch.clamp(1 - outputs.t() * labels, min=0)) / batch_size # modificar el 1 per la loss corresponent
def visualize_model(model):
    # Apartat B. Mostrar pesos del model
    plt.figure(0)
    #extraiem els pesos del model i els reformatajem en format imatge.
    plt.imshow(model.w.weight.detach().numpy().reshape(28, 28), interpolation='nearest', cmap=plt.cm.RdBu)
    plt.show()
    
def visualize_confusion_matrix(y_pred, y_real):
    #mostra la matriu de confusió
    cm = confusion_matrix(y_real, y_pred)
    plt.subplots(figsize=(10, 6))
    sns.heatmap(cm, annot = True, fmt = 'g')
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()


def train(model, device, train_loader, optimizer, criterion, epoch, log_interval):
    #Ens posem en mode entrenament.
    model.train()
    #iterem les dades en batches (o lots)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # en el cas de treballar en cuda o gpu, ara els hi posariem
        optimizer.zero_grad()  # posem el gradient a zero (important) ja que sino, s'aniria acumulant a cada iteració
        output = model(data)   # fem un predict del model amb les dades actuals
        loss = criterion(output.view_as(target), target.type_as(output))  # calculem el error obtingut
        loss.backward()             # li diem que calculi el gradient
        optimizer.step()            # actualitza els pesos amb el gradient calculat
        if batch_idx % log_interval == 0:  # cada log_interval loops mostrem informació
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            #visualize_model(model)
def test(model, device, test_loader, criterion, show_confusion_matrix=False):
    model.eval()     # ens posem en mode evaluació
    test_loss = 0    # variables per acumular resultats
    correct = 0
    all_preds = [] 
    all_outputs = [] 
    all_targets = []
    with torch.no_grad():  # li estem dient que estem en evaluacio, i que necesitem calcular el gradient aqui dins 
        for data, target in test_loader:   # per cada batch de test
            data, target = data.to(device), target.to(device)  # posar-ho a la cpu o a la gpu
            output = model(data)    # fer la prediccio
            test_loss += criterion(output.view_as(target), target.type_as(output)).item() * data.shape[0]  # acumulem error
            pred = (output > 0.5) * 1  # per comprobar la prediccio escollim el threshold a 0.5
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred)
            all_targets.extend(target)
            all_outputs.extend(output)

    test_loss /= len(test_loader.dataset) # fem la mitja de l'error del dataset
    if show_confusion_matrix:
        visualize_confusion_matrix(all_targets, all_preds)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    return all_outputs
# Definim un pipeline que se li aplicarà a les dades del dataset


# Aqui agafem les dades del propi torch. Si no les tenim, amb el parametre download ens el baixarem automaticament.
# Esculliu on voleu descarregar-vos les dades i intenteu de no anar-les replicant per totes les carpetes..


USING CUDA: False


In [27]:


target = pd.DataFrame(Y_train["round_winner"])
data_train = data_utils.TensorDataset(torch.Tensor(np.array(X_train_selected)), torch.Tensor(np.array(target)))
train_loader = data_utils.DataLoader(data_train, **train_kwargs)

target = pd.DataFrame(Y_test["round_winner"])
data_test = data_utils.TensorDataset(torch.Tensor(np.array(X_test_selected)), torch.Tensor(np.array(target)))
test_loader = data_utils.DataLoader(data_test, **test_kwargs)

model = Model().to(device)  # instanciació del model
optimizer = torch.optim.SGD(model.parameters(), lr=1,weight_decay=0.1)   # optimitzador (Apartat C. Aqui podem definir el weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)  # permet reduir el Learning rate

criterion = LogisticLoss()
#criterion = HingeLoss()
#criterion = torch.nn.BCELoss(reduction='mean')

for epoch in range(1, epochs + 1):
    # A cada epoca, fem un entrenament del model (es veu un cop cada exemple)
    print("LEARNING RATE USED: {}".format(scheduler.get_last_lr()))
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval)
    test(model, device, test_loader, criterion,show_confusion_matrix=False)
    scheduler.step()  # reduim el Learning Rate segons el scheduler StepLR que fa servir gamma

if save_model:
    torch.save(model.state_dict(), "mnist.pt")


LEARNING RATE USED: [1]
Train Epoch: 1 [0/97928 (0%)]	Loss: 69.680779
Train Epoch: 1 [5000/97928 (5%)]	Loss: 73.889954
Train Epoch: 1 [10000/97928 (10%)]	Loss: 75.205460
Train Epoch: 1 [15000/97928 (15%)]	Loss: 76.242340
Train Epoch: 1 [20000/97928 (20%)]	Loss: 76.370201
Train Epoch: 1 [25000/97928 (26%)]	Loss: 78.207947
Train Epoch: 1 [30000/97928 (31%)]	Loss: 79.742867
Train Epoch: 1 [35000/97928 (36%)]	Loss: 75.431351
Train Epoch: 1 [40000/97928 (41%)]	Loss: 79.833481
Train Epoch: 1 [45000/97928 (46%)]	Loss: 76.264008
Train Epoch: 1 [50000/97928 (51%)]	Loss: 78.730911
Train Epoch: 1 [55000/97928 (56%)]	Loss: 80.859467
Train Epoch: 1 [60000/97928 (61%)]	Loss: 79.244339
Train Epoch: 1 [65000/97928 (66%)]	Loss: 76.351234
Train Epoch: 1 [70000/97928 (71%)]	Loss: 77.368546
Train Epoch: 1 [75000/97928 (77%)]	Loss: 76.847626
Train Epoch: 1 [80000/97928 (82%)]	Loss: 76.309860
Train Epoch: 1 [85000/97928 (87%)]	Loss: 77.349098
Train Epoch: 1 [90000/97928 (92%)]	Loss: 78.822533
Train Epoch: 1

In [28]:
#criterion = LogisticLoss()
criterion = HingeLoss()
#criterion = torch.nn.BCELoss(reduction='mean')

for epoch in range(1, epochs + 1):
    # A cada epoca, fem un entrenament del model (es veu un cop cada exemple)
    print("LEARNING RATE USED: {}".format(scheduler.get_last_lr()))
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval)
    test(model, device, test_loader, criterion,show_confusion_matrix=False)
    scheduler.step()  # reduim el Learning Rate segons el scheduler StepLR que fa servir gamma

if save_model:
    torch.save(model.state_dict(), "mnist.pt")


LEARNING RATE USED: [0.16806999999999994]
Train Epoch: 1 [0/97928 (0%)]	Loss: 100.713356
Train Epoch: 1 [5000/97928 (5%)]	Loss: 99.720139
Train Epoch: 1 [10000/97928 (10%)]	Loss: 92.265038
Train Epoch: 1 [15000/97928 (15%)]	Loss: 101.535110
Train Epoch: 1 [20000/97928 (20%)]	Loss: 86.389015
Train Epoch: 1 [25000/97928 (26%)]	Loss: 103.763710
Train Epoch: 1 [30000/97928 (31%)]	Loss: 114.221512
Train Epoch: 1 [35000/97928 (36%)]	Loss: 98.968407
Train Epoch: 1 [40000/97928 (41%)]	Loss: 93.044739
Train Epoch: 1 [45000/97928 (46%)]	Loss: 105.309433
Train Epoch: 1 [50000/97928 (51%)]	Loss: 100.000000
Train Epoch: 1 [55000/97928 (56%)]	Loss: 99.521965
Train Epoch: 1 [60000/97928 (61%)]	Loss: 92.650604
Train Epoch: 1 [65000/97928 (66%)]	Loss: 100.000000
Train Epoch: 1 [70000/97928 (71%)]	Loss: 93.573494
Train Epoch: 1 [75000/97928 (77%)]	Loss: 97.709206
Train Epoch: 1 [80000/97928 (82%)]	Loss: 98.165573
Train Epoch: 1 [85000/97928 (87%)]	Loss: 101.804543
Train Epoch: 1 [90000/97928 (92%)]	Loss

In [29]:
#criterion = LogisticLoss()
#criterion = HingeLoss()
criterion = torch.nn.BCELoss(reduction='mean')

for epoch in range(1, epochs + 1):
    # A cada epoca, fem un entrenament del model (es veu un cop cada exemple)
    print("LEARNING RATE USED: {}".format(scheduler.get_last_lr()))
    train(model, device, train_loader, optimizer, criterion, epoch, log_interval)
    test(model, device, test_loader, criterion,show_confusion_matrix=False)
    scheduler.step()  # reduim el Learning Rate segons el scheduler StepLR que fa servir gamma

if save_model:
    torch.save(model.state_dict(), "mnist.pt")


LEARNING RATE USED: [0.028247524899999984]
Train Epoch: 1 [0/97928 (0%)]	Loss: 1.581788
Train Epoch: 1 [5000/97928 (5%)]	Loss: 0.729441
Train Epoch: 1 [10000/97928 (10%)]	Loss: 0.573056
Train Epoch: 1 [15000/97928 (15%)]	Loss: 0.641319
Train Epoch: 1 [20000/97928 (20%)]	Loss: 0.425861
Train Epoch: 1 [25000/97928 (26%)]	Loss: 0.508980
Train Epoch: 1 [30000/97928 (31%)]	Loss: 0.453294
Train Epoch: 1 [35000/97928 (36%)]	Loss: 0.421742
Train Epoch: 1 [40000/97928 (41%)]	Loss: 0.458444
Train Epoch: 1 [45000/97928 (46%)]	Loss: 0.495311
Train Epoch: 1 [50000/97928 (51%)]	Loss: 0.547342
Train Epoch: 1 [55000/97928 (56%)]	Loss: 0.441551
Train Epoch: 1 [60000/97928 (61%)]	Loss: 0.458410
Train Epoch: 1 [65000/97928 (66%)]	Loss: 0.512883
Train Epoch: 1 [70000/97928 (71%)]	Loss: 0.463834
Train Epoch: 1 [75000/97928 (77%)]	Loss: 0.497519
Train Epoch: 1 [80000/97928 (82%)]	Loss: 0.538224
Train Epoch: 1 [85000/97928 (87%)]	Loss: 0.541689
Train Epoch: 1 [90000/97928 (92%)]	Loss: 0.543323
Train Epoch: 1

SVM,hacer con las 30 variables

In [25]:
from sklearn.svm import SVC
import numpy as np
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#X_train,X_test,Y_train,Y_test = train_test_split(X, y, test_size=0.2,random_state =42)
lin_clf = svm.LinearSVC()#onevsone
lin_clf.fit(X_test_selected, Y_test.values.ravel())
y_pred=lin_clf.predict(X_test_selected)
# View The Accuracy Of Our Selected Feature Model
print("Accuracy",accuracy_score(Y_test, y_pred))

Accuracy 0.7493260354546197


C:\Users\oriol\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [26]:
from sklearn.neural_network import MLPClassifier 
model = MLPClassifier()
model.fit(X_train_selected, Y_train.values.ravel())

C:\Users\oriol\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [27]:
y_preds = model.predict(X_test_selected)

In [28]:
print("Accuracy",accuracy_score(Y_test, y_pred))

Accuracy 0.8563434359937914
